In [1]:
import os
import requests
import re
from datetime import datetime
import xml.etree.ElementTree as ET
from lxml import html
import nest_asyncio
nest_asyncio.apply()
from ib_insync import *
from yahoo_fin.stock_info import get_data
import pandas as pd
import pandas_ta as ta
from finvizfinance.quote import finvizfinance
from dateutil.relativedelta import relativedelta
from alpha_vantage.fundamentaldata import FundamentalData

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [2]:
API_KEY = 'fb9af3b1d45dda27119a970ab5f5a92c'

In [3]:
# Function to fetch stock price data
def fetch_stock_data(ticker):
    try:
        data = get_data(ticker)
        if 'close' not in data.columns:
            return pd.DataFrame()
        return data[['close']]
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

In [4]:
fetch_stock_data('NVDA')

,close
1999-01-22,0.041016
1999-01-25,0.045313
1999-01-26,0.041797
1999-01-27,0.041667
1999-01-28,0.041536
...,...
2024-07-05,125.830002
2024-07-08,128.199997
2024-07-09,131.380005
2024-07-10,134.910004


In [167]:
from datetime import datetime
# Function to fetch data from FinancialModelingPrep
def fetch_ownership_percent(ticker, API_KEY):
    url = f'https://financialmodelingprep.com/api/v4/institutional-ownership/symbol-ownership?symbol={ticker}&includeCurrentQuarter=true&apikey={API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        df = pd.DataFrame(data)
        df['date'] = pd.to_datetime(df['date'])
        ownership_percent = df.loc[:, ['date', "investorsHolding", 'ownershipPercent', 'ownershipPercentChange']]
        ownership_percent['3_month_diff'] = -df['ownershipPercent'].diff() 
        return ownership_percent
    else:
        print(f"Error: {response.status_code} for ticker {ticker}")
        return None

In [174]:
fetch_ownership_percent('OLLI', API_KEY)

,date,investorsHolding,ownershipPercent,ownershipPercentChange,3_month_diff
0,2024-03-31,386,100.5377,0.9966,NaN
1,2023-12-31,394,100.8831,0.9819,-0.3454
2,2023-09-30,373,102.7480,0.9977,-1.8649
3,2023-06-30,339,102.9811,0.9906,-0.2331
4,2023-03-31,339,103.9541,0.9909,-0.9730
5,2022-12-31,331,104.9047,0.9799,-0.9506
6,2022-09-30,329,107.0596,0.9937,-2.1549
7,2022-06-30,343,107.7393,0.9990,-0.6797
8,2022-03-31,300,107.8432,0.9879,-0.1039
9,2021-12-31,336,109.1639,0.9960,-1.3207


In [7]:
def fetch_institutional_ownership_data(ticker, fixed_date):
    # URL of the page to scrape
    url = f'https://fintel.io/so/us/{ticker}?d={fixed_date}'
    # Make the request to fetch the page content
    html_content = requests.get(url).content

    # Parse the page content using lxml
    tree = html.fromstring(html_content)

    # Use XPath to locate the data
    # This example assumes you want the first row's second column data
    xpath_expression = '//*[@id="main-content"]/div/div[1]/div/div/div[2]/table/tbody/tr[2]/td[2]/text()'
    amount = tree.xpath(xpath_expression)

    match = re.search(r'\$ ([\d,]+)', amount[0])
    institutional_value = float(match.group(1).replace(',', '')) *10**3

    return institutional_value

In [81]:
def calculate_institutional_shareholder_percentage(ticker, fixed_date):
    three_months_ago = datetime.strptime(fixed_date, '%Y-%m-%d') - relativedelta(months=3)
    three_months_ago = three_months_ago.strftime('%Y-%m-%d')
    print(three_months_ago)
    # Fetch the market capitalization data
    market_cap = fetch_market_cap(ticker, fixed_date, API_KEY)
    print(market_cap)
    past_market_cap = fetch_market_cap(ticker, three_months_ago, API_KEY)

    # Fetch the institutional ownership value
    institutional_value = fetch_institutional_ownership_data(ticker, fixed_date)
    print(institutional_value )

    past_institutional_value = fetch_institutional_ownership_data(ticker, three_months_ago)
    
    # Calculate the institutional shareholders percentage
    institutional_percentage = (institutional_value / market_cap) * 100
    print(institutional_percentage)
    past_institutional_percentage = (past_institutional_value / past_market_cap) * 100
    print(past_institutional_percentage)
    inst_trans =  (institutional_percentage/past_institutional_percentage-1) * 100
    
    return inst_trans

In [82]:
calculate_institutional_shareholder_percentage('NVDA', '2024-07-11')

2024-04-11
3137572800000.0
1465494908000.0
46.707917279242096
65.38403723352853


-28.563730146521817

In [94]:
# Function to fetch and parse fundamental data from Interactive Brokers
def fetch_fundamental_data(ticker):
    # Set up the connection parameters
    host = os.getenv('IB_HOST', 'host.docker.internal')
    port = int(os.getenv('IB_PORT', '4001'))
    ib = IB()
    ib.connect(host, port, clientId=1)

    contract = Stock(ticker, 'SMART', 'USD')
    
    fundamental_data = ib.reqFundamentalData(contract, 'ReportsOwnership')
    estimates_data = ib.reqFundamentalData(contract, 'RESC')
    print(fundamental_data)
    root = ET.fromstring(fundamental_data)
    rootx = ET.fromstring(estimates_data)

    ib.disconnect()
    return root, rootx

In [11]:
fetch_fundamental_data('NVDA')

NameError: name 'fetch_fundamental_data' is not defined

In [8]:
# Function to find the most recent and the second most recent value for a given period and date
def find_recent_values(root, tag, period, report_type, fixed_date):
    values = []
    for elem in root.findall(f'.//{tag}'):
        if elem.get('period') == period and elem.get('reportType') == report_type:
            date = datetime.strptime(elem.get('asofDate'), '%Y-%m-%d')
            if date <= fixed_date:
                values.append((date, float(elem.text)))
    values.sort(key=lambda x: x[0], reverse=True)
    return [values[0], values[4]]

# Function to extract EPS values
def extract_eps_values(rootx, year, value_type='ActValue'):
    if value_type == 'ActValue':
        for elem in rootx.findall('.//FYActual[@type="EPS"]//FYPeriod'):
            if elem.get('fYear') == str(year) and elem.get('periodType') == 'A':
                value_elem = elem.find('.//ActValue')
                if value_elem is not None:
                    return float(value_elem.text)
    elif value_type == 'ConsValue':
        for elem in rootx.findall('.//FYEstimate[@type="EPS"]//FYPeriod'):
            if elem.get('fYear') == str(year) and elem.get('periodType') == 'A':
                value_elem = elem.find('.//ConsEstimate[@type="Mean"]//ConsValue[@dateType="CURR"]')
                if value_elem is not None:
                    return float(value_elem.text)
    return None

In [48]:
# Function to calculate technical indicators
def calculate_indicators(data):
    data['SMA_20'] = data['close'].rolling(window=20).mean()
    data['SMA_50'] = data['close'].rolling(window=50).mean()
    data['SMA_200'] = data['close'].rolling(window=200).mean()
    data['RSI_14'] = ta.rsi(data['close'], length=14)
    return data

In [49]:
# Function to apply filters
def apply_filters(ticker, current_date):
    data = fetch_stock_data(ticker)
    if data.empty:
        return None

    current_datetime = pd.to_datetime(current_date)
    data = data[data.index <= current_datetime]  # Filter data up to the given date
    if data.empty:
        return None
    
    institutional_transactions = calculate_institutional_shareholder_percentage(ticker, current_datetime)
    print(institutional_transactions)
    if institutional_transactions <= 0:
        return None

    data = calculate_indicators(data)
    latest_close = data['close'].iloc[-1]
    sma20 = data['SMA_20'].iloc[-1]
    sma50 = data['SMA_50'].iloc[-1]
    sma200 = data['SMA_200'].iloc[-1]
    rsi14 = data['RSI_14'].iloc[-1]

    if latest_close <= 10:
        return None
    if rsi14 < 60:
        return None
    if latest_close < sma20 or latest_close < sma50 or latest_close < sma200:
        return None
    
    root, rootx, rooty = fetch_fundamental_data(ticker)
    
    # Extract EPS values for the relevant periods
    eps_3m_values = find_recent_values(root, 'EPS', '3M', 'A', current_datetime)

    # Extract revenue values for the relevant periods
    revenue_3m_values = find_recent_values(root, 'TotalRevenue', '3M', 'A', current_datetime)
    
    # Calculate EPS growth quarter over quarter (3M period)
    if len(eps_3m_values) == 2:
        eps_growth_qtr_qtr = (eps_3m_values[0][1] - eps_3m_values[1][1]) / abs(eps_3m_values[1][1]) * 100
        print("eps_growth_qtr_qtr", eps_growth_qtr_qtr)
        if eps_growth_qtr_qtr <= 20:
            return None

    # Calculate sales growth quarter over quarter (3M period)
    if len(revenue_3m_values) == 2:
        sales_growth_qtr_qtr = (revenue_3m_values[0][1] - revenue_3m_values[1][1]) / abs(revenue_3m_values[1][1]) * 100
        print("sales_growth_qtr_qtr", sales_growth_qtr_qtr)
        if sales_growth_qtr_qtr <= 0:
            return None

    eps_actual_current_year = extract_eps_values(rootx, current_datetime.year, value_type='ActValue')
    eps_estimate_next_year = extract_eps_values(rootx, current_datetime.year + 1, value_type='ConsValue')
    eps_estimate_year_after_next = extract_eps_values(rootx, current_datetime.year + 2, value_type='ConsValue')

    if eps_actual_current_year and eps_estimate_next_year:
        eps_growth_this_year = (eps_estimate_next_year - eps_actual_current_year) / abs(eps_actual_current_year) * 100
    elif current_datetime.year==datetime.now().year:
        match = re.search(r'(\d+\.?\d*)%', finvizfinance(ticker).ticker_fundament()['EPS this Y'])
        eps_growth_this_year = float(match.group(1))
    else:
        return None
    
    print("eps_growth_this_year", eps_growth_this_year)
    
    if eps_growth_this_year <= 20:
        return None

    if eps_estimate_next_year and eps_estimate_year_after_next:
        eps_growth_next_year = (eps_estimate_year_after_next - eps_estimate_next_year) / abs(eps_estimate_next_year) * 100
        print("eps_growth_next_year", eps_growth_next_year)
        if eps_growth_next_year <= 20:
            return None

    # Add more filters as necessary
    print("latest_close", latest_close)
    print("rsi14", rsi14)
    print("sma20", (latest_close/sma20-1)*100)
    print("sma50", (latest_close/sma50-1)*100)
    print("sma200", (latest_close/sma200-1)*100)
    print("institutional_transactions", institutional_transactions)
    return ticker

In [50]:
apply_filters('WMT', '2024-06-12')

[]


IndexError: list index out of range

In [1]:
!pip install refinitiv-data


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
import refinitiv.data as rd

In [3]:
rd.open_session(name='platform.rdp',
                config_name="./refinitiv-data.config.json")


/usr/local/lib/python3.9/site-packages/refinitiv/data/_access_layer/session.py:71:FutureWarning: 
You open a platform session using the default value of the signon_control parameter (signon_control=True).
In future library version v2.0, this default will be changed to False.
If you want to keep the same behavior as today, you will need to set the signon_control parameter to True either in the library configuration file
({'sessions':{'platform':{'your_session_name':{'signon_control':true}}}}) or in your code where you create the Platform Session.
These alternative options are already supported in the current version of the library.
[Error 400] - {'error': 'access_denied'}


<refinitiv.data.session.Definition object at 0xffffa1462760 {name='rdp'}>

In [183]:
google_revenues = rd.get_history(
        universe="GOOG.O",
        fields=["TR.Revenue"],
        interval="1Y",
        start="2015-01-01",
        end="2024-01-01",
    )

RDError: Error code -1 | Authorization header is missing Requested universes: ['GOOG.O']. Requested fields: ['TR.REVENUE']